In [1]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def load_and_process_file(file_path):
    # Load a game file and return as a sequence of features
    data = pd.read_csv(file_path, header=None)
    data = data.iloc[:12, 1:]
    return data.values.T  # Return as a 2D numpy array

def load_data(features_directory, labels_file_path):
    features_list = []
    labels = np.loadtxt(labels_file_path)
    
    for filename in sorted(os.listdir(features_directory)):
        if filename.endswith(".txt"):
            file_path = os.path.join(features_directory, filename)
            features = load_and_process_file(file_path)
            features_list.append(features)
    
    # Pad sequences so that they are all the same length
    # features_padded = pad_sequences(features_list, padding='post', dtype='float32')
    return features_list, labels

# Paths to the data
features_directory = './data/'
labels_file_path = './labels/game_state.txt'


In [2]:

# Load and prepare data
features_array, labels_array = load_data(features_directory, labels_file_path)

# print(features_array)
print(labels_array)
unique_values, value_counts = np.unique(labels_array, return_counts=True)
print(unique_values, value_counts)


[2. 0. 1. ... 2. 2. 2.]
[0. 1. 2.] [509 724 742]


In [4]:


from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical


X_train_, X_test, y_train_, y_test = train_test_split(features_array, labels_array, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_val = train_test_split(X_train_, y_train_, test_size=0.1, random_state=42)
# n_features = X_train.shape[2]
y_train = to_categorical(y_train)

In [7]:
X_train_padded = pad_sequences(X_train_, padding='post', value=-1)

In [8]:
n_features = 12
n_classes = 3

In [20]:
model = Sequential()
model.add(Masking(mask_value=-1, input_shape=(None, n_features)))
model.add(LSTM(200, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_, epochs=200, validation_split=0.2)

Epoch 1/200


KeyboardInterrupt: 